In [1]:
from arcgis.gis import GIS
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
import json

In [2]:
gis = GIS(profile = "eowilson")

In [3]:
import re
def create_strict_reg_exp(to_search):
    try:
        reg_exp = f"^{to_search}$"
        #logging.info(f"regular expression is: {reg_exp}")
    except:
        print("There was a problem with the string.")
    return reg_exp

In [4]:
def findItemGetID(csvName, gis):
    try:
        searched_item = gis.content.search(csvName, item_type = "Feature Layer")
        for i in searched_item:
            reg_exp = create_strict_reg_exp(csvName)
            if re.search(reg_exp, i.title)!= None:    
                #logging.info(f"{csvName} has the id: {i.id}")
                return i.id
    except:
        print("There was a problem finding the item")

In [5]:
def getFlayerFromID(item_id):
    item = gis.content.get(item_id)
    flayer = item.layers[0]
    return flayer

In [6]:
import pandas as pd
def getSDFfromFlayer(flayer):
    sdf = pd.DataFrame.spatial.from_layer(flayer)
    return sdf

# Get the item and get an SDF

In [7]:
item_id = findItemGetID("gadm_centroid", gis)

In [8]:
flayer = getFlayerFromID(item_id)

In [9]:
sdf = getSDFfromFlayer(flayer)

In [13]:
sdf.head()

/opt/anaconda3/lib/python3.7/site-packages/IPython/lib/pretty.py:402: FutureWarning: 'ExtensionArray._formatting_values' is deprecated. Specify 'ExtensionArray._formatter' instead.
  return _repr_pprint(obj, self, cycle)
/opt/anaconda3/lib/python3.7/site-packages/pandas/io/formats/html.py:606: FutureWarning: 'ExtensionArray._formatting_values' is deprecated. Specify 'ExtensionArray._formatter' instead.
  super().render()


,OBJECTID_1,GID_0,NAME_0,jpg_url,OBJECTID,GID,Area,GNI_PPP,Protected,HM_0,...,endemic_mammals,endemic_reptiles,protection_needed,spi_mean,HM_very_high,prop_hm_very_high,GlobalID,continent,filter_similar,SHAPE
0,1,ABW,Aruba,https://live.staticflickr.com/1952/31416683438...,1,ABW,175.0,3.9,0.0,0.0,...,0,3,78.07,56.880635,33.0,18.857143,fe9f6eb0-f4f8-4f29-875a-5cbb3219e4e5,North America,"{""filter_Area"": ""[\""JEY\"", \""CXR\"", \""WLF\"", \...","{""x"": -69.97027575642096, ""y"": 12.509315424467..."
1,2,AFG,Afghanistan,https://p1.pxfuel.com/preview/967/12/53/afghan...,2,AFG,643780.0,70.6,596.0,815.0,...,0,4,46.87,56.880635,334192.0,51.910901,193ba976-0e5a-4cf6-9b09-d00bf83f4557,Asia,"{""filter_Area"": ""[\""MDG\"", \""UKR\"", \""CAF\"", \...","{""x"": 66.02958641598036, ""y"": 33.8284153570830..."
2,3,AGO,Angola,https://live.staticflickr.com/3787/13698381215...,3,AGO,1247225.0,192.2,4726.0,3.0,...,9,23,55.72,56.880635,541523.0,43.418228,174ce788-4f67-4ae0-922f-d2ddac87f8c3,Africa,"{""filter_Area"": ""[\""BOL\"", \""ETH\"", \""COL\"", \...","{""x"": 17.578063914717916, ""y"": -12.33829953999..."
3,4,AIA,Anguilla,https://live.staticflickr.com/8063/8194570372_...,4,AIA,79.0,NaN,0.0,0.0,...,0,1,84.47,56.880635,0.0,0.000000,9f5f24d8-8b21-49a8-8f55-90b47cf63e7b,North America,"{""filter_Area"": ""[\""PCN\"", \""MAF\"", \""BVT\"", \...","{""x"": -63.05439849309596, ""y"": 18.214736293445..."
4,5,ALA,Åland,https://p1.pxfuel.com/preview/294/670/561/alan...,5,ALA,1265.0,NaN,15.0,23.0,...,0,0,1.49,56.880635,1242.0,98.181818,2b45351b-a335-490e-914e-7748d4f41f66,Europe,"{""filter_Area"": ""[\""KIR\"", \""TCA\"", \""STP\"", \...","{""x"": 19.966666219770275, ""y"": 60.240625385353..."


# Get nearest countries
The sdf only has the centroids of the countries. We could calculate the distance between all the points: 255 x 255 matrix and then keep the top 20 of closest. Check [this resource](https://kanoki.org/2019/12/27/how-to-calculate-distance-in-python-and-pandas-using-scipy-spatial-and-distance-functions/) to calculate distance. 

In [34]:
from math import radians
import pandas as pd
import numpy as np
from sklearn.neighbors import DistanceMetric

In [25]:
df_coord = pd.DataFrame(data = sdf['GID_0'])

In [27]:
y_list = []
x_list = []
for index, i in sdf.iterrows():
    y_list.append(i['SHAPE'].y)
    x_list.append(i['SHAPE'].x)

In [28]:
df_coord['x'] = x_list
df_coord['y'] = y_list

In [32]:
df_coord['lat'] = np.radians(df_coord['y'])
df_coord['lon'] = np.radians(df_coord['x'])

In [35]:
dist = DistanceMetric.get_metric('haversine')

In [37]:
dist_df = pd.DataFrame(dist.pairwise(df_coord[['lat','lon']].to_numpy())*6373,  columns=df_coord.GID_0.unique(), index=df_coord.GID_0.unique())

In [39]:
dist_df.shape

(252, 252)

Next is to sort the table for each point, 0 will be the same country. So we take the 1:11 and add the row names to a dictionary and boom we have the names of the countries. 

In [63]:
neighbour_dict = dict.fromkeys(dist_df_sort.columns.values)

In [66]:
dist_df_sort = dist_df
neighbour_dict = dict.fromkeys(dist_df_sort.columns.values)
for key in neighbour_dict:
    dist_df_sort = dist_df_sort.sort_values(by = [key])
    # here i have the df sorted by one country
    # i have to take for each loop the 1:11 slice, collapse it and add it to the dictionary using the GID as key.
    sub = dist_df_sort[field][0:11]
    vals = sub.index.values.tolist()
    neighbour_dict
    neighbour_dict[key] = json.dumps(vals)
    break
neighbour_dict    

{'ABW': '["ABW", "CUW", "BES", "DOM", "HTI", "VEN", "PRI", "VIR", "VGB", "GRD", "KNA"]',
 'AFG': None,
 'AGO': None,
 'AIA': None,
 'ALA': None,
 'ALB': None,
 'AND': None,
 'ARE': None,
 'ARG': None,
 'ARM': None,
 'ASM': None,
 'ATA': None,
 'ATF': None,
 'ATG': None,
 'AUS': None,
 'AUT': None,
 'AZE': None,
 'BDI': None,
 'BEL': None,
 'BEN': None,
 'BES': None,
 'BFA': None,
 'BGD': None,
 'BGR': None,
 'BHR': None,
 'BHS': None,
 'BIH': None,
 'BLM': None,
 'BLR': None,
 'BLZ': None,
 'BMU': None,
 'BOL': None,
 'BRA': None,
 'BRB': None,
 'BRN': None,
 'BTN': None,
 'BVT': None,
 'BWA': None,
 'CAF': None,
 'CAN': None,
 'CCK': None,
 'CHE': None,
 'CHL': None,
 'CHN': None,
 'CIV': None,
 'CMR': None,
 'COD': None,
 'COG': None,
 'COK': None,
 'COL': None,
 'COM': None,
 'CPV': None,
 'CRI': None,
 'CUB': None,
 'CUW': None,
 'CXR': None,
 'CYM': None,
 'CYP': None,
 'CZE': None,
 'DEU': None,
 'DJI': None,
 'DMA': None,
 'DNK': None,
 'DOM': None,
 'DZA': None,
 'ECU': None,
 

# Get below and above countries for each field in challenges

In [15]:
fields = ['GID_0', 'NAME_0', 'Area', 'GNI_PPP', 'Population2016', 'prop_protected', 'prop_hm_very_high', 'protection_needed', 'total_endemic', 'N_SPECIES', 'SPI']

In [16]:
df = sdf[fields]

In [89]:
filter_fields = ['Area','GNI_PPP','Population2016', 'prop_protected', 'prop_hm_very_high', 'protection_needed', 'total_endemic','N_SPECIES', 'SPI']

In [179]:
nber_index = 5
max_index = len(df_sort.index) - 1
df_sort = df
new_fields = []
for field in filter_fields:
    df_sort = df_sort.sort_values(by = [field]).reset_index(drop=True)
    collapse_list = []
    for index, i in df_sort.iterrows():
        country_gid = df_sort.GID_0[index]
        above_index = index - nber_index
        below_index = index + nber_index + 1
        if above_index < 0:
            below_index = nber_index * 2 
            above_index = 0
        if below_index > max_index:
            above_index = max_index - (nber_index * 2)
            below_index = max_index

        sub_pd = df_sort.GID_0[above_index:below_index]
        val_list = sub_pd.values.tolist()
        collapse_list.append(json.dumps(val_list))

        #val_list_rem = val_list.remove(country_gid)
    filter_field = f"filter_{field}"
    new_fields.append(filter_field)
    df_sort[filter_field] = collapse_list

In [180]:
similar_list = []
for index, i in df_sort.iterrows():
    filter_dict = i[new_fields].to_dict()
    similar_list.append(json.dumps(filter_dict))
    #similar_list.append(json.loads(json.dumps(filter_dict)))    
df_sort['filter_similar'] = similar_list

In [181]:
df_upload = df_sort[["GID_0", "filter_similar"]]

# Update service


In [167]:
from copy import deepcopy
def createFieldsToBeAdded(flayer, csv_table):
    flayer_fields = flayer.manager.properties.fields
    template_field = dict(deepcopy(flayer_fields[0]))
    sdf = getSDFfromFlayer(flayer)
    new_field_names = list(csv_table.columns.difference(sdf.columns))
    
    fields_to_be_added = []
    for new_field_name in new_field_names:
        current_field = deepcopy(template_field)
        dt = csv_table[new_field_name].dtypes
        
        if dt == 'O':
            #put the type to character
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeString'
            #current_field['length'] = 8000
        if dt == 'float64':
            #put the type to double
            current_field['sqlType'] = 'sqlTypeOther'
            current_field['type'] = 'esriFieldTypeDouble'
            #current_field['length'] = 8000      

        current_field['name'] = new_field_name.lower()
        current_field['alias'] = new_field_name
        current_field['nullable'] = True
        current_field['editable'] = True
        fields_to_be_added.append(current_field)
    return fields_to_be_added

In [141]:
def createFeaturesForUpdate(flayer, csv_table, fields_to_be_added, id_field_in_csv, id_field_in_service):
    fset2 = flayer.query()
    features2 = fset2.features
    features_for_update = []
    for country_id in csv_table[id_field_in_csv]:
        try:
            # get the matching row from csv
            matching_row = csv_table.where(csv_table[id_field_in_csv] == country_id).dropna()

            #print(str(country_id) + " Adding additional attributes for: " + matching_row['iso3'].values[0])

            # get the feature to be updated
            assert  len([f for f in features2 if f.attributes[id_field_in_service] == country_id]),  "id not matched"
            original_feature = [f for f in features2 if f.attributes[id_field_in_service] == country_id][0]
            feature_to_be_updated = deepcopy(original_feature)

            # assign the updated values
            for field in fields_to_be_added:
                feature_to_be_updated.attributes[field['name']] = matching_row[field['name']].values[0]
                #add this to the list of features to be updated
                features_for_update.append(feature_to_be_updated)
            #print(str(country_id) + " Done additional attributes for: " + matching_row['countryname'].values[0])
    
        except:
            print(f"{country_id} not available in service")
    return features_for_update

In [182]:
fields_to_be_added = createFieldsToBeAdded(flayer, df_upload)

In [183]:
fields_to_be_added

[{'name': 'filter_similar',
  'type': 'esriFieldTypeString',
  'alias': 'filter_similar',
  'sqlType': 'sqlTypeOther',
  'nullable': True,
  'editable': True,
  'visible': True,
  'domain': None,
  'defaultValue': None}]

In [184]:
flayer.manager.add_to_definition({'fields':fields_to_be_added})

{'success': True}

In [185]:
features_for_update = createFeaturesForUpdate(flayer, df_upload, fields_to_be_added, "GID_0", "GID_0")

In [186]:
flayer.edit_features(updates= features_for_update)

{'addResults': [],
 'updateResults': [{'objectId': 41,
   'uniqueId': 41,
   'globalId': '66165570-9dda-43b4-8f3e-ed0f8c2bd38f',
   'success': True},
  {'objectId': 183,
   'uniqueId': 183,
   'globalId': '105c59d6-ac35-4756-ace3-a120d431cba4',
   'success': True},
  {'objectId': 13,
   'uniqueId': 13,
   'globalId': '3fc7e6c9-dbc2-4c37-b09a-947f046237eb',
   'success': True},
  {'objectId': 51,
   'uniqueId': 51,
   'globalId': '7b10595e-8e8f-483a-b9e2-fcbc823a72d4',
   'success': True},
  {'objectId': 25,
   'uniqueId': 25,
   'globalId': '31946c78-0b11-4005-8519-73dc2a6073f8',
   'success': True},
  {'objectId': 201,
   'uniqueId': 201,
   'globalId': 'a9876c34-b478-4322-a56b-93d25af3d87c',
   'success': True},
  {'objectId': 49,
   'uniqueId': 49,
   'globalId': 'e412b775-d873-4c7d-b18a-b22fa400b895',
   'success': True},
  {'objectId': 120,
   'uniqueId': 120,
   'globalId': '66b06090-9d4a-4a8e-8715-68fc12b14f30',
   'success': True},
  {'objectId': 82,
   'uniqueId': 82,
   'glob